In [2]:
import numpy as np
import pandas as pd

# Вспомним (напишем) KNN для классификации

In [3]:
#Считаем данные из файла data/Howell1.csv в DataFrame
# Должно быть 4 колонки: height, weight, age, male
df = pd.read_csv("data/Howell1.csv", sep=";")
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head()

,height,weight,age,male
0,156.845,53.041914,41.0,1
1,163.830,62.992589,35.0,1
2,149.225,38.243476,32.0,0
3,168.910,55.479971,27.0,1
4,147.955,34.869885,19.0,0


In [4]:
#Разобьем наши данные на трейн и валидацию (не будем делать cross-validation, возьмем просто отложенный семпл)
#Чем плох такой подход?
#Используем train_test_split с random_state=13, test_size=0.2 и стратификацией по целевой переменной 
#ПРЕДСКАЗЫВАТЬ БУДЕМ ПОЛ!
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.drop("male", axis=1), df['male'],
                                                    random_state=13, test_size=0.2, stratify=df['male'])

In [6]:
#Что еще нужно сделать для KNN?

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
#Заполните пропуски в классификаторе
class KNNClassifier:
    """
    simple KNN classifier
    """
    
    def __init__(self, n):
        self.n = n
        
    def fit(self, X, y):
        self.X = X
        self.y = y
    
    def predict(self, X):
        y = []
        assert len(X.shape) == 2
        for (h, w, a) in X:
            ### Посчитаем расстояние от всех элементов в тренировочной выборке
            # до текущего примера -> результат - вектор размерности трейна
            d = np.sqrt((h - self.X[:, 0])**2 + (w - self.X[:, 1])**2 + (a - self.X[:, 2])**2)
            ### Возьмем индексы n элементов, расстояние до которых минимально
            ### результат -> вектор из n элементов
            idx = np.argsort(d)[:self.n]
            ### Посчитаем частоту меток для каждого случая 
            ### результат -> вектор длинны 2 который покажет
            ### сколько соседей 0-го класса, сколько соседей 1-го класса.
            counts = np.average(self.y[idx])
            ### возьмем самый часто встречаемый в соседях класс.
            prediction = (counts > .5).astype('int64')
            y.append(prediction)
        return y

In [8]:
from sklearn.metrics import precision_score

In [9]:
#Подберем на валидации число соседей
possible_n = [3, 5, 7, 9, 13]
knnc = KNNClassifier(1)
knnc.fit(X_train_scaled, y_train.values)
for n in possible_n:
    knnc.n = n
    preds = knnc.predict(X_test_scaled)
    print(f'n = {n}, score = {precision_score(y_test.values, preds)}')
    
#TODO Подберем число соседей
#Хотим быть очень точными ;)

n = 3, score = 0.85
n = 5, score = 0.782608695652174
n = 7, score = 0.8571428571428571
n = 9, score = 0.8571428571428571
n = 13, score = 0.85


In [11]:
from sklearn.neighbors import KNeighborsClassifier

In [12]:
possible_n = [3, 5, 7, 9, 13]

for n in possible_n:
    knnc = KNeighborsClassifier(n_neighbors=n)
    knnc.fit(X_train_scaled, y_train.values)
    preds = knnc.predict(X_test_scaled)
    print(f'n = {n}, score = {precision_score(y_test.values, preds)}')
    

n = 3, score = 0.85
n = 5, score = 0.782608695652174
n = 7, score = 0.8571428571428571
n = 9, score = 0.8571428571428571
n = 13, score = 0.85


# KNN для регрессии

In [17]:
#TODO Препроцессинг данных для регрессии. Целевая переменная - ВОЗРАСТ, посмотрим можно ли что-то сказать
#о возрасте по весу, росту и полу (Hint, скорее всего нет)
class KNNRegressor:
    """
    simple KNN regressor
    """
    
    def __init__(self, n):
        self.n = n
        
    def fit(self, X, y):
        self.X = X
        self.y = y
    
    def predict(self, X):
        y = []
        assert len(X.shape) == 2
        for (w, a) in X:
            ### Посчитаем расстояние от всех элементов в тренировочной выборке
            # до текущего примера -> результат - вектор размерности трейна
            d = np.sqrt((w - self.X[:, 0])**2 + (a - self.X[:, 1])**2)
            ### Возьмем индексы n элементов, расстояние до которых минимально
            ### результат -> вектор из n элементов
            idx = np.argsort(d)[:self.n]
            prediction = np.average(self.y[idx])
            y.append(prediction)
        return y

In [18]:
X_train_reg = X_train[['weight', 'age']]
y_train_reg = X_train['height']
X_test_reg = X_test[['weight', 'age']]
y_test_reg = X_test['height']

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_reg_scaled = scaler.fit_transform(X_train_reg)
X_test_reg_scaled = scaler.transform(X_test_reg)

In [21]:
from sklearn.metrics import r2_score

In [22]:
#Подберем на валидации число соседей
possible_n = [3, 5, 7, 9, 13]
knnc = KNNRegressor(1)
knnc.fit(X_train_reg_scaled, y_train_reg.values)
for n in possible_n:
    knnc.n = n
    preds = knnc.predict(X_test_reg_scaled)
    print(f'n = {n}, score = {r2_score(y_test_reg.values, preds)}')

n = 3, score = 0.45278903209058496
n = 5, score = 0.5903592986428168
n = 7, score = 0.6247632177427449
n = 9, score = 0.6094000064071579
n = 13, score = 0.6069134022814966


In [23]:
from sklearn.neighbors import KNeighborsRegressor

In [24]:
possible_n = [3, 5, 7, 9, 13]

for n in possible_n:
    knnc = KNeighborsRegressor(n_neighbors=n)
    knnc.fit(X_train_reg_scaled, y_train_reg.values)
    preds = knnc.predict(X_test_reg_scaled)
    print(f'n = {n}, score = {r2_score(y_test_reg.values, preds)}')
    

n = 3, score = 0.45278903209058496
n = 5, score = 0.5903592986428168
n = 7, score = 0.6247632177427449
n = 9, score = 0.6065896025378983
n = 13, score = 0.6069134022814966
